In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
from torch import nn

import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset, random_split
from torchvision import transforms
import glob
import os

/home/csgrad/byalavar/miniconda3/envs/train/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [49]:
# import zipfile

# def unzip_file(zip_file_path, extract_to):
#     with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#         zip_ref.extractall(extract_to)

# # Example usage
# zip_file_path = '/home/csgrad/byalavar/FHE/HEAAN/FG_Round_2/face-samples.zip'
# extract_to_directory = '/home/csgrad/byalavar/FHE/HEAAN/FG_Round_2/face-samples'

# unzip_file(zip_file_path, extract_to_directory)


In [4]:
import os
import csv

# Function to extract information from the image file name and grandparent folder names
def extract_info(file_path):
    parts = file_path.split(os.sep)
    filename = parts[-1].split('.')[0]  # Get the filename without extension
    #gender = parts[-2]
    ethnicity = parts[-3].split('_')[0]
    gender = parts[-3].split('_')[1]
    print(filename,gender,ethnicity)
    return filename, gender, ethnicity,file_path

# Function to process a directory and create a CSV file
def process_directory(directory_path, output_csv):
    with open(output_csv, 'w', newline='') as csvfile:
        fieldnames = ['Name', 'Gender', 'Ethnicity','Path']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for root, dirs, files in os.walk(directory_path):
            for file in files:
                file_path = os.path.join(root, file)
                name, gender, ethnicity,path = extract_info(file_path)
                writer.writerow({'Name': name, 'Gender': gender, 'Ethnicity': ethnicity,'Path':path})

# Example usage
dataset_directory = '/home/csgrad/byalavar/FHE/HEAAN/FG_Round_2/face-samples'
output_csv_file = '/home/csgrad/byalavar/FHE/HEAAN/FG_Round_2/bfw.csv'

process_directory(dataset_directory, output_csv_file)


0067_01 males indian
0268_01 males indian
0388_01 males indian
0372_01 males indian
0256_02 males indian
0069_01 males indian
0167_01 males indian
0295_01 males indian
0034_01 males indian
0103_01 males indian
0098_01 males indian
0077_01 males indian
0005_01 males indian
0164_01 males indian
0321_01 males indian
0196_01 males indian
0241_01 males indian
0014_01 males indian
0236_01 males indian
0145_02 males indian
0012_01 males indian
0139_01 males indian
0158_01 males indian
0218_01 males indian
0110_01 males indian
0036_01 males indian
0206_01 males indian
0041_01 males indian
0148_01 males indian
0038_01 males indian
0463_01 males indian
0171_01 males indian
0423_01 males indian
0245_01 males indian
0155_01 males indian
0103_01 males indian
0022_01 males indian
0187_01 males indian
0112_01 males indian
0019_01 males indian
0193_01 males indian
0057_01 males indian
0283_01 males indian
0428_01 males indian
0005_01 males indian
0237_01 males indian
0254_04 males indian
0196_01 males

In [14]:
import torch
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:2


In [3]:
#!wget https://hydranets-data.s3.eu-west-3.amazonaws.com/UTKFace.zip

In [4]:
#!unzip UTKFace.zip

In [6]:


class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.gender_mapping = {'male': 0, 'female': 1}
        self.ethnicity_mapping = {'white': 0, 'black': 1, 'asian': 2, 'hispanic': 3}


    def __len__(self):
        return len(self.data)


    
    def getAgeLabel(self,value1):

    # Define class ranges
        class_ranges = [(0, 5),(5,13),(13,20),(20,40),(40,60),(60,120)]
    
    # Check if both values fall into the same class range
        if(class_ranges[0][0]<=value1 and value1<class_ranges[0][1]):
            return 0
        elif(class_ranges[1][0]<=value1 and value1<class_ranges[1][1]):
            return 1
        elif(class_ranges[2][0]<=value1 and value1<class_ranges[2][1]):
            return 2
        elif(class_ranges[3][0]<=value1 and value1<class_ranges[3][1]):
            return 3
        elif(class_ranges[4][0]<=value1 and value1<class_ranges[4][1]):
            return 4
        elif(class_ranges[5][0]<=value1 and value1<class_ranges[5][1]):
            return 5
        # elif(class_ranges[6][0]<=value1 and value1<class_ranges[6][1]):
        #     return 6
        # elif(class_ranges[7][0]<=value1 and value1<class_ranges[7][1]):
        #     return 7
#        else:
 #           return 0
   

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = '/home/csgrad/byalavar/FHE/celebSet/CELEBTEST/CELEBTEST/'+row['name']+'/' + row['filename']  # Assuming images are in a folder named 'images'
        
        try:
            image = Image.open(image_path)
        except Exception as e:
            # Handle the error, for example, you can return a placeholder image
            print("here")
            #self.__getitem__(idx + 1)
            #image = Image.new('RGB', (224, 224))  # Create a blank image
        
        image = Image.open(image_path)
        age = row['age']
        
        if(row['age']<=0):
            age=35
        label = {
            'age': self.getAgeLabel(age),
            'gender': self.gender_mapping.get(row['gender'], 0),  # -1 for unknown
            'ethnicity': self.ethnicity_mapping.get(row['ethnicity'], 0),
            'age1':age
        
        }
        #print(row['name'])
        if self.transform:
            image = self.transform(image)

        return image, label



In [10]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.gender_mapping = {'males': 0, 'females': 1}
        self.ethnicity_mapping = {'indian': 0, 'white': 1, 'asian': 2, 'black': 3}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]['Path']
        image = Image.open(img_path).convert('RGB')


        gender = self.data.iloc[idx]['Gender']
        ethnicity = self.data.iloc[idx]['Ethnicity']
        age = self.data.iloc[idx]['predictedAge']

        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

        label = {
            'age': age,
            'gender': self.gender_mapping.get(gender, 0),  # -1 for unknown
            'ethnicity': self.ethnicity_mapping.get(ethnicity, 0),
        
        }
        return image,label

# Example usage:
csv_file_path = '/home/csgrad/byalavar/FHE/HEAAN/FG_Round_2/bfw.csv'  # Replace with the actual path to your CSV file
transform = transforms.Compose([
    transforms.Resize((112, 112)),  # Resize the image to the desired size
    transforms.ToTensor(),          # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])


dataset = CustomDataset(csv_file_path, transform=transform)

In [7]:
# csv_file = '/home/csgrad/byalavar/FHE/celebSet/celebSET_final_v1.csv'  # Replace with the actual path to your CSV file
# df = pd.read_csv(csv_file)

# # Create a list to store the indices of rows with missing files
# rows_to_remove = []
# count=0
# # Iterate through the DataFrame and check if the files exist
# for index, row in df.iterrows():
#     image_path = '/home/csgrad/byalavar/FHE/celebSet/CELEBTEST/CELEBTEST/'+row['name']+'/' + row['filename'] 
#     if not os.path.exists(image_path):
#         rows_to_remove.append(index)
#         count=count+1
# df = df.drop(rows_to_remove)
# df.to_csv(csv_file, index=False)
# print(count)

0


In [11]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create dataloaders for training and testing sets
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [8]:
next(iter(train_dataloader))

[tensor([[[[ 1.6667,  1.5810,  1.4440,  ..., -0.0458, -0.0287, -0.0629],
           [ 1.6153,  1.5125,  1.3413,  ..., -0.2171, -0.2171, -0.2513],
           [ 1.4612,  1.2728,  1.1529,  ..., -0.2856, -0.1999, -0.1657],
           ...,
           [ 0.2796,  0.3481,  0.3823,  ...,  0.2796,  0.3481,  0.4337],
           [ 0.1083,  0.2111,  0.2967,  ...,  0.3481,  0.3138,  0.2967],
           [-0.0801,  0.0912,  0.2111,  ...,  0.8104,  0.8104,  0.7419]],
 
          [[-0.1625, -0.1099, -0.0749,  ..., -0.8102, -0.8102, -0.8452],
           [-0.2150, -0.1975, -0.1625,  ..., -0.9678, -1.0028, -1.0553],
           [-0.3550, -0.4251, -0.3725,  ..., -1.0378, -0.9853, -0.9503],
           ...,
           [-0.5301, -0.4601, -0.4251,  ...,  0.0651,  0.1527,  0.2577],
           [-0.7052, -0.6001, -0.5126,  ...,  0.2052,  0.2052,  0.1877],
           [-0.8978, -0.7227, -0.6001,  ...,  0.7129,  0.7304,  0.6779]],
 
          [[-0.4101, -0.4450, -0.5147,  ..., -1.4036, -1.4036, -1.4384],
           [-

In [13]:
def imshow(img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        plt.show()

In [49]:
import math
import numpy
def generate_C(C_range, m):
    """ Randomly generates m coefficients for the PolyProtect mapping.

    **Inputs:**

    C_range : integer
        The absolute min/max values of the coefficients range.

    m : int
        The number of coefficients to generate.

    **Outputs:**

    C : 1D numpy array of integers
        Array of m coefficients.

    """

    # Calculate coefficient range (excluding 0):
    neg_range = numpy.arange(-1 * C_range, 0)
    pos_range = numpy.arange(1, C_range + 1)
    whole_range = numpy.concatenate([neg_range, pos_range])

    # Randomly generate m unique coefficients:
    C = numpy.random.permutation(whole_range)[0 : m] # randomly permute the whole range and pick the first few m values

    return C


def generate_E(m):
    """ Randomly generates m exponents for the PolyProtect mapping.

    **Inputs:**

    m : int
        The number of exponents to generate.

    **Outputs:**

    E : 1D numpy array of integers
        Array of m exponents.

    """

    # Randomly generate m unique exponents:
    E = numpy.random.permutation(range(1, m + 1))[0 : m] # permute the integers in the range [1, m]

    return E


def polyprotect(overlap, V):
    """ Maps an embedding to a PolyProtected template.

    **Inputs:**

    overlap : int
        The amount of overlap between sets of embedding elements used to generate each PolyProtected element (0, 1, 2, 3, or 4).

    V : torch.Tensor
        The embedding as a PyTorch tensor.

    **Outputs:**

    P : torch.Tensor
        The PolyProtected template as a PyTorch tensor.

    """

    C = torch.tensor([-42, -35, 31, 4], dtype=torch.float32, device=V.device)
    E = torch.tensor([3, 2, 1, 4], dtype=torch.float32, device=V.device)

    if C.shape[0] != E.shape[0]:
        print("Number of coefficients and exponents must be the same.")
        return None
    #print("here")
    m = C.shape[0] # number of embedding elements used to generate each PolyProtected element
    step_size = m - overlap
    decimal_remainder, integer = math.modf((V.shape[0] - m) / step_size)
    if decimal_remainder > 0:
        padding = math.ceil((1 - decimal_remainder) * step_size)
    else:
        padding = 0
   # print("here1")
    # Pad V by "padding" zeros at the end
    V = torch.cat((V, torch.zeros(padding, device=V.device)), dim=0)

    starting_indices = torch.arange(0, V.shape[0] - m + 1, step_size)
    #print("here2")
    P = torch.zeros(len(starting_indices), device=V.device)
    
    for storage_ind, ind in enumerate(starting_indices):
        
        final_ind = ind + m
        crnt_word = V[ind:final_ind]
        P[storage_ind] = torch.sum(C * (crnt_word ** E))
    #print("here3")
    return P

In [56]:


class faceAnalytics(nn.Module):

    def __init__(self,var):
        super().__init__()
        self.var = var
        self.layer1=nn.Linear(var,256)
        self.dropout1=nn.Dropout(0.2)
        self.layer2=nn.Linear(256,128)
        #self.layer3=nn.Linear(1024,512)
        self.layer4=nn.Linear(128,64)
        self.dropout2=nn.Dropout(0.2)
        self.genderOut=nn.Linear(64,2)
        self.ageOut=nn.Linear(64,6)
        self.ethnicityOut = nn.Linear(64,4)

        # self.maxVal = 0
        # self.min=0
        
    
    def writeResult(self,result):
       output_directory=""
       file_name = "resultAge.txt"

       with open(os.path.join(output_directory, file_name), "w") as file:
        for value in result:
            file.write(f"{value}\n")
    
    def forward(self,x):

        #print("Input",x[0])
        x=self.layer1(x)
        #print(x[0])
        #x=nn.functional.relu(x)
        x=self.dropout1(x)
        #x=nn.functional.relu(x)
        x=self.layer2(x)
        x=nn.functional.relu(x)
        #self.writeResult(x[0])
       # x=self.layer3(x)
        x=self.layer4(x)
        #print(torch.max(torch.abs(x)))
        x=nn.functional.relu(x)
        
        x=self.dropout2(x)
        gender=self.genderOut(x)
        
        age=self.ageOut(x)

        ethn = self.ethnicityOut(x)
        #self.writeResult(age[0])
        #print(gender)
        return gender,age,ethn
    
        
    
    #tempPT,genderLabelTensor,ageLabelTensor,ethnLabelTensor
    def trainModel(self,tempPT,genderLabelTensor,ageLabelTensor,ethnLabelTensor,testLoader,device,episodes,resnet):
        
        #maxVal = 0
        learningRate=0.005
        gender_loss = nn.CrossEntropyLoss() 
        age_loss = nn.CrossEntropyLoss()
        ethn_loss = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=learningRate,weight_decay=0.001)
        #ageLabelTensor = torch.where(ageLabelTensor == 6, torch.tensor(5), ageLabelTensor)

        #processedTensor = torch.zeros((len(train_dataset),128)).to(device)
        #print(processedTensor.shape)
 #processedTensor = torch.zeros((len(train_dataloader.dataset),128))
        # ageLabelTensor = torch.zeros((len(train_dataset))).to(device)
        # genderLabelTensor = torch.zeros((len(train_dataset))).to(device)
        # ethnLabelTensor = torch.zeros((len(train_dataset))).to(device)
        for e in range(0,episodes):
         total_training_loss =0
         tempAccGender=0
         tempAccAge=0
         tempAccEthn=0
         count=0
         totalGenderLoss=0
         self.train()
         
      

        #     inputs = data["image"].to(device=device)
        #     #inputs = inputs.type(torch.double)
        #     age_label = data["age"].to(device=device)
        #     gender_label = data["gender"].to(device=device)
        #     ethn_label = data["ethn"].to(device=device)
         batchSize = 256

         ageLabelTensor = ageLabelTensor.type(torch.LongTensor)
         genderLabelTensor = genderLabelTensor.type(torch.LongTensor)
         ethnLabelTensor = ethnLabelTensor.type(torch.LongTensor)    

         while(count<tempPT.shape[0]):
            if(count+batchSize<=tempPT.shape[0]):
             inputs = tempPT[count:count+batchSize].to(device=device)
             age_label = ageLabelTensor[count:count+batchSize].to(device=device)
             gender_label = genderLabelTensor[count:count+batchSize].to(device=device)
             ethn_label = ethnLabelTensor[count:count+batchSize].to(device=device)
            else:
               inputs = tempPT[count:].to(device=device)
               age_label = ageLabelTensor[count:].to(device=device)
               gender_label = genderLabelTensor[count:].to(device=device)
               ethn_label = ethnLabelTensor[count:].to(device=device)
            
            
    
            gender,age,ethn = self(inputs)
            # age=torch.squeeze(age)
            # age=age.type(torch.float32)
            #print(age.shape,age_label.shape)
            #print(gender.shape,gender_label.shape)
            # predictedGender = torch.argmax(gender,dim=1)
            # predictedGender = predictedGender.type(torch.float32)
            #print(gender)
            loss =   5*age_loss(age,age_label) + 4*ethn_loss(ethn,ethn_label) + gender_loss(gender,gender_label)
            #totalGenderLoss = totalGenderLoss + loss.item()
            loss.backward()
            #print("Loss:",loss)
            optimizer.step()
            optimizer.zero_grad()
            total_training_loss = total_training_loss+loss.item()*256

            predictedGender = torch.argmax(gender,dim=1)
            
            
            for j in range(0,predictedGender.shape[0]):
                count=count+1
                #print(predictedGender[j].item(),gender_label[j])
                if(predictedGender[j].item()==gender_label[j]):
                    tempAccGender=tempAccGender+1
            
            predictedAge = torch.argmax(age,dim=1)
            
            for j in range(0,predictedAge.shape[0]):
               
                #print(predictedGender[j].item(),gender_label[j])
                if(predictedAge[j].item()==age_label[j]):
                    tempAccAge=tempAccAge+1
            
            predictedEthn = torch.argmax(ethn,dim=1)
            
            for j in range(0,predictedEthn.shape[0]):
               
                #print(predictedGender[j].item(),gender_label[j])
                if(predictedEthn[j].item()==ethn_label[j]):
                    tempAccEthn=tempAccEthn+1
         
            genderAccuracy =  tempAccGender/count
            ageAccuracy =  tempAccAge/count
            ethnAccuracy =  tempAccEthn/count
            #print(count)
            #break
         #break;
         print("Training Gender Accuracy:", genderAccuracy,"Age:", ageAccuracy,"Ethn:",ethnAccuracy)
        #  print("\n")
        #  print("Training Ethn Accuracy:", ethnAccuracy,"\n")
        #  print("total training loss:",total_training_loss/16595,"\n")
         #print("\n")
         #print("max observed value: ", maxVal)
        #  if(e%2==0):
        #      self.test(testLoader,device)

        # print(processedTensor.shape,processedTensor[0])
        # print(ageLabelTensor.shape,ageLabelTensor[0])
        # print(genderLabelTensor.shape,genderLabelTensor[0])
        # print(ethnLabelTensor.shape,ethnLabelTensor[0])
        #return processedTensor,genderLabelTensor,ageLabelTensor,ethnLabelTensor


    def are_values_in_same_class(self,value1, value2):

    # Define class ranges
        class_ranges = [(0, 2),(2,5),(5,13),(13,20),(20,40),(40,60),(60,80),(80,120)]
    
    # Check if both values fall into the same class range
        for class_range in class_ranges:
            if class_range[0] <= value1 <= class_range[1] and class_range[0] <= value2 <= class_range[1]:
                return True
    
        return False

    def test(self,testLoader,resnet,device):

        predGender = []
        predAge = []
        predEthn = []
        labGender = []
        labAge = []
        labEthn = []
        tempAccGender=0
        tempAccAge=0
        tempAccEthn=0
        count=0

        self.eval()
         
        for i,data in enumerate(testLoader):

            inputs = data[0].to(device=device)
            #inputs = inputs.type(torch.double)
            age_label = data[1]["age"].to(device=device)
            gender_label = data[1]["gender"].to(device=device)
            ethn_label = data[1]["ethnicity"].to(device=device)
            
            
            inputs=resnet(inputs)
            #print(inputs.shape)
            inputUpdated = torch.zeros((inputs.shape[0], 128),device=device)
            for t in range(0,inputs.shape[0]):
             #print(inputs[t].shape,"inputs[t]")   
             inputUpdated[t] = polyprotect(0,inputs[t])
            #print("here4")
            #print(inputUpdated[0])
            gender,age,ethn = self(inputUpdated)
            
            predictedGender = torch.argmax(gender,dim=1)
            for j in range(0,predictedGender.shape[0]):
                count=count+1
                predGender.append(predictedGender[j].item())
                labGender.append(gender_label[j].item())
                #print(predictedGender[j].item(),gender_label[j])
                if(predictedGender[j].item()==gender_label[j]):
                    tempAccGender=tempAccGender+1
            
            predictedAge = torch.argmax(age,dim=1)
            for j in range(0,predictedAge.shape[0]):
                predAge.append(predictedAge[j].item())
                labAge.append(age_label[j].item())
                #print(predictedGender[j].item(),gender_label[j])
                if(predictedAge[j].item()==age_label[j]):
                    tempAccAge=tempAccAge+1
            
            predictedEthn = torch.argmax(ethn,dim=1)
            for j in range(0,predictedEthn.shape[0]):
                predEthn.append(predictedEthn[j].item())
                labEthn.append(ethn_label[j].item())
                #print(predictedGender[j].item(),gender_label[j])
                if(predictedEthn[j].item()==ethn_label[j]):
                    tempAccEthn=tempAccEthn+1
         
        genderAccuracy =  tempAccGender/count
        ageAccuracy =  tempAccAge/count
        ethnAccuracy =  tempAccEthn/count
        
        print("Test Gender Accuracy:", genderAccuracy,"\n")
        print("Test Age Accuracy:", ageAccuracy,"\n")
        print("Test Ethn Accuracy:", ethnAccuracy,"\n")
   
         #print("\n")
        return predGender,predAge,predEthn,labGender,labAge,labEthn
            
       

In [22]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
# count=0
# for param in resnet.parameters():
#     count=count+1
#     if(count>350):
#      param.requires_grad = True
#     else:
#      param.requires_grad = False

# print(count)

In [39]:
device

device(type='cuda', index=0)

In [57]:
var = 128
model=faceAnalytics(var)
#model=torch.load("modelUsing0.pt")
model.to(device)
processedTensor = torch.load("inputTempBFW.pt")
processedTensor = processedTensor.to(device)
# #print(processedTensor.shape)
genderLabelTensor= torch.load("genderTempBFW.pt")
genderLabelTensor = genderLabelTensor.to(device)
ageLabelTensor = torch.load("ageTempBFW.pt")
ageLabelTensor = ageLabelTensor.to(device)
ethnLabelTensor = torch.load("ethnTempBFW.pt")
ethnLabelTensor = ethnLabelTensor.to(device)
model.trainModel(processedTensor,genderLabelTensor,ageLabelTensor,ethnLabelTensor,test_dataloader,device,20,resnet)

#processedTensor,genderLabelTensor,ageLabelTensor,ethnLabelTensor = model.trainModel(train_dataloader,val_dataloader,device,1,resnet)
# #torch.save(model,"faTest14.pt")
# torch.save(processedTensor,"inputUTK.pt")
# torch.save(genderLabelTensor,"genderLabelUTK.pt")
# torch.save(ageLabelTensor,"ageLabelUTK.pt")
# torch.save(ethnLabelTensor,"ethnLabelUTK.pt")


Training Gender Accuracy: 0.8136875 Age: 0.8651875 Ethn: 0.82925
Training Gender Accuracy: 0.9221875 Age: 0.9053125 Ethn: 0.8996875
Training Gender Accuracy: 0.9348125 Age: 0.9116875 Ethn: 0.9025
Training Gender Accuracy: 0.9335625 Age: 0.916375 Ethn: 0.9063125
Training Gender Accuracy: 0.9410625 Age: 0.9203125 Ethn: 0.9083125
Training Gender Accuracy: 0.94125 Age: 0.9224375 Ethn: 0.9133125
Training Gender Accuracy: 0.94075 Age: 0.924375 Ethn: 0.9155
Training Gender Accuracy: 0.9430625 Age: 0.925375 Ethn: 0.9169375
Training Gender Accuracy: 0.943625 Age: 0.928 Ethn: 0.917875
Training Gender Accuracy: 0.9438125 Age: 0.9266875 Ethn: 0.919125
Training Gender Accuracy: 0.9465 Age: 0.932 Ethn: 0.9185
Training Gender Accuracy: 0.943625 Age: 0.931625 Ethn: 0.924125
Training Gender Accuracy: 0.94425 Age: 0.9305625 Ethn: 0.919625
Training Gender Accuracy: 0.9454375 Age: 0.9305625 Ethn: 0.92275
Training Gender Accuracy: 0.9444375 Age: 0.9339375 Ethn: 0.922625
Training Gender Accuracy: 0.9436875 

In [44]:
torch.save(model,"fa3CS.pt")

In [58]:
#model=faceAnalytics()
#model=torch.load("fa3CS.pt")
# model.to(device)
#print(model)
#(0.9170416197975253, 0.7315804274465691, 0.8307086614173228)
predGender,predAge,predEthn,labGender,labAge,labEthn = model.test(test_dataloader,resnet,device)

Test Gender Accuracy: 0.95075 

Test Age Accuracy: 0.93975 

Test Ethn Accuracy: 0.91625 



In [46]:
labGender

[1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,


In [48]:
from sklearn.metrics import classification_report
report = classification_report(labAge, predAge)
print(report)

              precision    recall  f1-score   support

           2       0.50      0.11      0.18         9
           3       0.96      0.98      0.97      3299
           4       0.88      0.80      0.84       688
           5       0.00      0.00      0.00         4

    accuracy                           0.94      4000
   macro avg       0.58      0.47      0.50      4000
weighted avg       0.94      0.94      0.94      4000



/home/csgrad/byalavar/miniconda3/envs/train/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/csgrad/byalavar/miniconda3/envs/train/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/csgrad/byalavar/miniconda3/envs/train/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

              precision    recall  f1-score   support

           0       0.90      0.89      0.90       996
           1       0.92      0.91      0.91       996
           2       0.94      0.94      0.94      1034
           3       0.91      0.94      0.93       974

    accuracy                           0.92      4000
   macro avg       0.92      0.92      0.92      4000
weighted avg       0.92      0.92      0.92      4000



In [50]:
# Input Extract code



class faceAnalytics(nn.Module):

    def __init__(self,var):
        super().__init__()
        self.var = var
        self.layer1=nn.Linear(var,256)
        self.dropout1=nn.Dropout(0.2)
        self.layer2=nn.Linear(256,128)
        #self.layer3=nn.Linear(1024,512)
        self.layer4=nn.Linear(128,64)
        self.dropout2=nn.Dropout(0.2)
        self.genderOut=nn.Linear(64,2)
        self.ageOut=nn.Linear(64,6)
        self.ethnicityOut = nn.Linear(64,4)

        # self.maxVal = 0
        # self.min=0
        
    
    def writeResult(self,result):
       output_directory=""
       file_name = "resultAge.txt"

       with open(os.path.join(output_directory, file_name), "w") as file:
        for value in result:
            file.write(f"{value}\n")
    
    def forward(self,x):

        #print("Input",x[0])
        x=self.layer1(x)
        #print(x[0])
        #x=nn.functional.relu(x)
        #x=self.dropout1(x)
        #x=nn.functional.relu(x)
        x=self.layer2(x)
        
        #self.writeResult(x[0])
       # x=self.layer3(x)
        x=self.layer4(x)
        #print(torch.max(torch.abs(x)))
        x=nn.functional.relu(x)
        
        x=self.dropout2(x)
        gender=self.genderOut(x)
        
        age=self.ageOut(x)

        ethn = self.ethnicityOut(x)
        #self.writeResult(age[0])
        #print(gender)
        return gender,age,ethn
    
        
    
    #tempPT,genderLabelTensor,ageLabelTensor,ethnLabelTensor
    def trainModel(self,trainLoader,testLoader,device,episodes,resnet):
        
        #maxVal = 0
        learningRate=0.005
        gender_loss = nn.CrossEntropyLoss() 
        age_loss = nn.CrossEntropyLoss()
        ethn_loss = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=learningRate,weight_decay=0.003)
        processedTensor = torch.zeros((len(trainLoader.dataset),self.var)).to(device)
        print(processedTensor.shape)
 #processedTensor = torch.zeros((len(train_dataloader.dataset),128))
        ageLabelTensor = torch.zeros((len(trainLoader.dataset))).to(device)
        genderLabelTensor = torch.zeros((len(trainLoader.dataset))).to(device)
        ethnLabelTensor = torch.zeros((len(trainLoader.dataset))).to(device)
        for e in range(0,episodes):
         total_training_loss =0
         tempAccGender=0
         tempAccAge=0
         tempAccEthn=0
         count=0
         totalGenderLoss=0
         self.train()
         
         for i,data in enumerate(trainLoader):

            inputs = data[0].to(device=device)
            #inputs = inputs.type(torch.double)
            age_label = data[1]["age"].to(device=device)
            gender_label = data[1]["gender"].to(device=device)
            ethn_label = data[1]["ethnicity"].to(device=device)
         #batchSize = 256

        #  ageLabelTensor = ageLabelTensor.type(torch.LongTensor)
        #  genderLabelTensor = genderLabelTensor.type(torch.LongTensor)
        #  ethnLabelTensor = ethnLabelTensor.type(torch.LongTensor)    

        #  while(count<tempPT.shape[0]):
        #     if(count+batchSize<=tempPT.shape[0]):
        #      inputs = tempPT[count:count+batchSize].to(device=device)
        #      age_label = ageLabelTensor[count:count+batchSize].to(device=device)
        #      gender_label = genderLabelTensor[count:count+batchSize].to(device=device)
        #      ethn_label = ethnLabelTensor[count:count+batchSize].to(device=device)
        #     else:
        #        inputs = tempPT[count:].to(device=device)
        #        age_label = ageLabelTensor[count:].to(device=device)
        #        gender_label = genderLabelTensor[count:].to(device=device)
        #        ethn_label = ethnLabelTensor[count:].to(device=device)
            
            
            inputs=resnet(inputs)
            # #print(inputs.shape)
            inputUpdated = torch.zeros((inputs.shape[0], 128),device=device)
            for t in range(0,inputs.shape[0]):
             #print(inputs[t].shape,"inputs[t]")   
             inputUpdated[t] = polyprotect(0,inputs[t])
             #count = count +1
            #print("here4")
             
            #print(inputUpdated[0]) 
            #print(inputUpdated[1])
            processedTensor[count:count+inputUpdated.shape[0]] = inputUpdated
            ageLabelTensor[count:count+age_label.shape[0]] = age_label
            genderLabelTensor[count:count+gender_label.shape[0]] = gender_label
            ethnLabelTensor[count:count+ethn_label.shape[0]] = ethn_label
            #continue
            gender,age,ethn = self(inputUpdated)
            # age=torch.squeeze(age)
            # age=age.type(torch.float32)
            #print(age.shape,age_label.shape)
            #print(gender.shape,gender_label.shape)
            # predictedGender = torch.argmax(gender,dim=1)
            # predictedGender = predictedGender.type(torch.float32)
            #print(gender)
            loss = gender_loss(gender,gender_label) + 4*age_loss(age,age_label) + 4*ethn_loss(ethn,ethn_label)
            #totalGenderLoss = totalGenderLoss + loss.item()
            loss.backward()
            #print("Loss:",loss)
            optimizer.step()
            optimizer.zero_grad()
            total_training_loss = total_training_loss+loss.item()*256

            predictedGender = torch.argmax(gender,dim=1)
            
            
            for j in range(0,predictedGender.shape[0]):
                count=count+1
                #print(predictedGender[j].item(),gender_label[j])
                if(predictedGender[j].item()==gender_label[j]):
                    tempAccGender=tempAccGender+1
            
            predictedAge = torch.argmax(age,dim=1)
            
            for j in range(0,predictedAge.shape[0]):
               
                #print(predictedGender[j].item(),gender_label[j])
                if(predictedAge[j].item()==age_label[j]):
                    tempAccAge=tempAccAge+1
            
            predictedEthn = torch.argmax(ethn,dim=1)
            
            for j in range(0,predictedEthn.shape[0]):
               
                #print(predictedGender[j].item(),gender_label[j])
                if(predictedEthn[j].item()==ethn_label[j]):
                    tempAccEthn=tempAccEthn+1
         
            genderAccuracy =  tempAccGender/count
            ageAccuracy =  tempAccAge/count
            ethnAccuracy =  tempAccEthn/count
            #print(count)
            #break
         break;
         print("Training Gender Accuracy:", genderAccuracy,"Age:", ageAccuracy,"Ethn:",ethnAccuracy)
        #  print("\n")
        #  print("Training Ethn Accuracy:", ethnAccuracy,"\n")
        #  print("total training loss:",total_training_loss/16595,"\n")
         #print("\n")
         #print("max observed value: ", maxVal)
        #  if(e%2==0):
        #      self.test(testLoader,device)

        # print(processedTensor.shape,processedTensor[0])
        # print(ageLabelTensor.shape,ageLabelTensor[0])
        # print(genderLabelTensor.shape,genderLabelTensor[0])
        # print(ethnLabelTensor.shape,ethnLabelTensor[0])
        return processedTensor,genderLabelTensor,ageLabelTensor,ethnLabelTensor


    def are_values_in_same_class(self,value1, value2):

    # Define class ranges
        class_ranges = [(0, 2),(2,5),(5,13),(13,20),(20,40),(40,60),(60,80),(80,120)]
    
    # Check if both values fall into the same class range
        for class_range in class_ranges:
            if class_range[0] <= value1 <= class_range[1] and class_range[0] <= value2 <= class_range[1]:
                return True
    
        return False

    def test(self,testLoader,resnet,device):

    
        tempAccGender=0
        tempAccAge=0
        tempAccEthn=0
        count=0

        self.eval()
         
        for i,data in enumerate(testLoader):

            inputs = data["image"].to(device=device)
            #inputs = inputs.type(torch.double)
            age_label = data["age"].to(device=device)
            gender_label = data["gender"].to(device=device)
            ethn_label = data["ethnicity"].to(device=device)
            
            
            inputs=resnet(inputs)
            #print(inputs.shape)
            inputUpdated = torch.zeros((inputs.shape[0], 128),device=device)
            for t in range(0,inputs.shape[0]):
             #print(inputs[t].shape,"inputs[t]")   
             inputUpdated[t] = polyprotect(0,inputs[t])
            #print("here4")
            #print(inputUpdated[0])
            gender,age,ethn = self(inputUpdated)
            
            predictedGender = torch.argmax(gender,dim=1)
            for j in range(0,predictedGender.shape[0]):
                count=count+1
                #print(predictedGender[j].item(),gender_label[j])
                if(predictedGender[j].item()==gender_label[j]):
                    tempAccGender=tempAccGender+1
            
            predictedAge = torch.argmax(age,dim=1)
            for j in range(0,predictedAge.shape[0]):
               
                #print(predictedGender[j].item(),gender_label[j])
                if(predictedAge[j].item()==age_label[j]):
                    tempAccAge=tempAccAge+1
            
            predictedEthn = torch.argmax(ethn,dim=1)
            for j in range(0,predictedEthn.shape[0]):
               
                #print(predictedGender[j].item(),gender_label[j])
                if(predictedEthn[j].item()==ethn_label[j]):
                    tempAccEthn=tempAccEthn+1
         
        genderAccuracy =  tempAccGender/count
        ageAccuracy =  tempAccAge/count
        ethnAccuracy =  tempAccEthn/count
        
        print("Test Gender Accuracy:", genderAccuracy,"\n")
        print("Test Age Accuracy:", ageAccuracy,"\n")
        print("Test Ethn Accuracy:", ethnAccuracy,"\n")
   
         #print("\n")
        return genderAccuracy,ageAccuracy, ethnAccuracy
            
       

In [51]:
var = 128
model=faceAnalytics(var)
model.to(device)
processedTensor,genderLabelTensor,ageLabelTensor,ethnLabelTensor = model.trainModel(train_dataloader,test_dataloader,device,1,resnet)

# torch.save(processedTensor,"inputCS.pt")
# torch.save(genderLabelTensor,"genderLabelCS.pt")
# torch.save(ageLabelTensor,"ageLabelCS.pt")
# torch.save(ethnLabelTensor,"ethnLabelCS.pt")

torch.Size([16000, 128])


torch.Size([16000, 512])

In [52]:
torch.save(processedTensor,"inputTempBFW.pt")
torch.save(genderLabelTensor,"genderTempBFW.pt")
torch.save(ageLabelTensor,"ageTempBFW.pt")
torch.save(ethnLabelTensor,"ethnTempBFW.pt")

In [ ]:
# Function to predict age from an image path

# def predict_age(image_path):
#     image = Image.open(image_path).convert('RGB')
#     preprocess = transforms.Compose([
#     transforms.Resize((112, 112)),  # Resize the image to the desired size
#     transforms.ToTensor(),          # Convert the image to a tensor
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
# ])
#     image = preprocess(image)
#     image = torch.unsqueeze(image, 0)
#     image = image.to(device)
#     print(image_path)
#     with torch.no_grad():
#         image = resnet(image)
#         age_prediction = model(image)
#         #print(age_prediction)
#         age_prediction = torch.argmax(age_prediction,dim=1).item()
#         #print(age_prediction)
#     return age_prediction
    
# # Predict ages and add labels to the CSV
# df['predictedAge'] = df['Path'].apply(predict_age)

# # Save the updated CSV file
# df.to_csv('bfw.csv', index=False)


In [53]:
file_path = 'input1.txt'

# Read data from the .txt file into a NumPy array
data = np.loadtxt(file_path, delimiter=',')

# Convert the NumPy array to a PyTorch tensor
tensor_data = torch.tensor(data).to(device)
tensor_data = tensor_data.type(torch.float32)
tensor_data = tensor_data.reshape(1,-1)

In [54]:
tensor_data.shape

torch.Size([1, 128])

In [66]:
model=faceAnalytics()
model=torch.load("modelUsing0.pt")
model.to(device)

model.test(val_dataloader,device,tensor_data)


In [36]:
model=faceAnalytics()
#model=torch.load("bestFaceAn.pt")
model.to(device)
#model.test(val_dataloader,device) # batchSize = 128, learning rate = 0.05
model.trainModel(train_dataloader,val_dataloader,device,8,resnet)
torch.save(model,"bestFaceAn.pt")

Training Gender Accuracy: 0.8246142719382835 

total training loss: 0.4848069211954667 



Test Gender Accuracy: 0.890466816647919 

Test Age Loss: 0.1393999291753608 



Training Gender Accuracy: 0.8826542912246866 

total training loss: 0.346408403792558 



Training Gender Accuracy: 0.8980834136933462 

total training loss: 0.3051145996663828 



Test Gender Accuracy: 0.9147919010123734 

Test Age Loss: 0.13645052078470202 



Training Gender Accuracy: 0.9035679845708775 

total training loss: 0.2846982805942261 



Training Gender Accuracy: 0.9057377049180327 

total training loss: 0.2736512154547268 



Test Gender Accuracy: 0.921541057367829 

Test Age Loss: 0.1309188806836329 



Training Gender Accuracy: 0.9090525554484089 

total training loss: 0.2645829449291867 



Training Gender Accuracy: 0.9110414657666345 

total training loss: 0.25597767612954947 



Test Gender Accuracy: 0.9246344206974129 

Test Age Loss: 0.1358024146076799 



Training Gender Accuracy: 0.909896335583

In [23]:
torch.save(model,"model92.pt")

In [24]:
count=0
input1=[]
for i,data in enumerate(val_dataloader):
    
    if(count==0):
     inputs=resnet(data["image"].to(device))

 
     input1 = polyprotect(0,inputs[0])

     break
    count=count+1

In [25]:
input1

tensor([-1.4430, -1.9958, -0.6684,  1.0280, -1.1776,  0.5820,  1.1919, -1.2120,
         0.5127, -0.1822,  0.6574, -1.9295,  1.2562, -1.0405, -1.5425, -2.9240,
         0.8135,  1.8771,  0.7530, -0.4737,  2.3692, -1.3006, -1.7764, -1.7158,
        -1.2634,  0.7241,  1.4031, -2.1193,  0.3556,  0.8521, -2.6465, -1.6996,
        -0.3830,  0.5889, -1.5234,  0.5707, -0.4224, -1.3535, -1.2482, -2.5428,
        -0.5712, -0.3220,  0.3134,  1.0675, -2.0277, -0.0984,  0.9127, -0.8203,
         0.5162,  0.8371,  0.2184,  1.7868,  0.1441,  2.2517,  0.9456,  0.2534,
        -2.4333, -1.6649,  0.5323, -0.8450,  0.6767, -1.2998,  0.1268, -1.4707,
        -1.2234, -3.1957, -0.3958,  2.0446, -1.4466,  1.2428, -0.1448,  1.3427,
        -0.7067, -1.0655, -1.0468, -4.1246,  1.0056, -1.1438, -0.4887,  1.7819,
        -0.3919, -1.5383,  3.3167,  0.0240, -0.7870, -2.7450, -2.3516, -0.2413,
        -1.3294, -2.1587, -1.5163,  1.6668, -0.6842, -1.0367, -1.9168,  0.2893,
        -1.6461, -0.0207, -0.1686,  1.37

In [30]:
for param in model.parameters():
    print(param.shape)
    print(param[0])
    print(torch.dot(input1,param[1]))
    break 

torch.Size([512, 128])
tensor([-0.0765, -0.1069, -0.0411,  0.0238,  0.0118,  0.0784, -0.0193, -0.0194,
        -0.0132,  0.0487,  0.0409, -0.0441, -0.1003,  0.0050,  0.0179,  0.0378,
        -0.0322,  0.0842, -0.0395, -0.0201,  0.0457, -0.0654, -0.0586,  0.0188,
        -0.0772,  0.0069,  0.0861, -0.0244,  0.0486, -0.0309, -0.0071,  0.0655,
        -0.1047,  0.0400,  0.0215, -0.0451,  0.0729, -0.0582, -0.0285,  0.0711,
        -0.0588, -0.0468, -0.0742, -0.0929,  0.0281,  0.0036, -0.0665, -0.0713,
        -0.0775,  0.0966,  0.0213,  0.0536, -0.0497, -0.0261,  0.0736, -0.0707,
        -0.0635,  0.0039,  0.0356,  0.0208, -0.0432,  0.0132,  0.0090, -0.0508,
         0.0649, -0.0474,  0.0852,  0.0400,  0.0783,  0.0792, -0.0682, -0.0400,
         0.0074, -0.0067,  0.0327,  0.0179, -0.0077, -0.0110,  0.0008,  0.0626,
         0.0268, -0.0395, -0.0766,  0.0686, -0.0601,  0.0466,  0.0655,  0.0477,
        -0.0761,  0.0306,  0.0702, -0.0810,  0.0598,  0.0180, -0.0715, -0.0503,
        -0.0228, 

In [69]:
input1

tensor([-1.4430, -1.9958, -0.6684,  1.0280, -1.1776,  0.5820,  1.1919, -1.2120,
         0.5127, -0.1822,  0.6574, -1.9295,  1.2562, -1.0405, -1.5425, -2.9240,
         0.8135,  1.8771,  0.7530, -0.4737,  2.3692, -1.3006, -1.7764, -1.7158,
        -1.2634,  0.7241,  1.4031, -2.1193,  0.3556,  0.8521, -2.6465, -1.6996,
        -0.3830,  0.5889, -1.5234,  0.5707, -0.4224, -1.3535, -1.2482, -2.5428,
        -0.5712, -0.3220,  0.3134,  1.0675, -2.0277, -0.0984,  0.9127, -0.8203,
         0.5162,  0.8371,  0.2184,  1.7868,  0.1441,  2.2517,  0.9456,  0.2534,
        -2.4333, -1.6649,  0.5323, -0.8450,  0.6767, -1.2998,  0.1268, -1.4707,
        -1.2234, -3.1957, -0.3958,  2.0446, -1.4466,  1.2428, -0.1448,  1.3427,
        -0.7067, -1.0655, -1.0468, -4.1246,  1.0056, -1.1438, -0.4887,  1.7819,
        -0.3919, -1.5383,  3.3167,  0.0240, -0.7870, -2.7450, -2.3516, -0.2413,
        -1.3294, -2.1587, -1.5163,  1.6668, -0.6842, -1.0367, -1.9168,  0.2893,
        -1.6461, -0.0207, -0.1686,  1.37

In [72]:
output_directory=""
file_name = "input1.txt"

with open(os.path.join(output_directory, file_name), "w") as file:
        for value in input1:
            file.write(f"{value}\n")

In [35]:
#model=torch.load("/home/csgrad/byalavar/HEAAN/bestFaceAn.pt")
model.test(val_dataloader,device)

TypeError: test() missing 1 required positional argument: 'tensor_data'

In [38]:
torch.save(model,"modelUsing0.pt")

In [82]:
count=0
for param in model.parameters():
    print(param.shape)
    count=count+1
    if(count==2):
        print(param)

torch.Size([512, 128])
torch.Size([512])
Parameter containing:
tensor([-0.0608, -0.0217,  0.0529, -0.0089,  0.0091,  0.0046, -0.0850,  0.0737,
         0.0143,  0.0217, -0.0977,  0.0810, -0.0564,  0.0011,  0.0722, -0.0247,
         0.0400, -0.0504, -0.0266,  0.0041,  0.0306, -0.0060, -0.0088, -0.0187,
         0.0724, -0.0828, -0.0848, -0.0345, -0.0290,  0.0105,  0.0145, -0.0039,
        -0.0052, -0.0191,  0.0698, -0.0620, -0.0487, -0.1063,  0.0467, -0.0002,
         0.0537,  0.0358, -0.0329,  0.0654, -0.0006, -0.1011, -0.0192, -0.0640,
        -0.0694,  0.0320, -0.1085,  0.0418, -0.0238, -0.0615, -0.0560,  0.0657,
        -0.0512,  0.0451, -0.0391,  0.0244, -0.0777, -0.0340,  0.0475,  0.0886,
         0.0003, -0.0109, -0.0577, -0.0619, -0.0323, -0.0008, -0.0176,  0.0066,
         0.0504,  0.0278, -0.0135,  0.0303,  0.0313, -0.0311,  0.0308, -0.0487,
         0.0603, -0.0842,  0.0063, -0.0263,  0.0686,  0.0789,  0.0931,  0.0091,
        -0.0183, -0.0787,  0.0457,  0.0954,  0.0281, -0.0

In [52]:
a=nn.Linear(4,2)
input1=torch.rand((1,4))
print("input1",input1)
for param in a.parameters():
    print("param",param)
print(a(input1))


input1 tensor([[0.1725, 0.7784, 0.7692, 0.3667]])
param Parameter containing:
tensor([[ 0.0794, -0.2791,  0.0171, -0.2814],
        [-0.0677, -0.3197, -0.2683, -0.3466]], requires_grad=True)
param Parameter containing:
tensor([ 0.0877, -0.3602], requires_grad=True)
tensor([[-0.2058, -0.9542]], grad_fn=<AddmmBackward0>)


In [70]:
model=faceAnalytics()
model=torch.load("modelUsing0.pt")
model.to(device)
count=0
ageBias=[]
for param in model.parameters(): 
    print(param.shape)
    if(count==7):
       ageBias = param.tolist()
    count=count+1
print(len(ageBias),len(ageBias[0]))


torch.Size([512, 128])
torch.Size([512])
torch.Size([128, 512])
torch.Size([128])
torch.Size([2, 128])
torch.Size([2])
torch.Size([1, 128])
torch.Size([1])


TypeError: object of type 'float' has no len()

In [71]:
ageBias

[0.09601110219955444]

In [41]:
bias2

[0.007277209311723709,
 -0.0321158841252327,
 -0.031418390572071075,
 -0.006457424722611904,
 0.0034283681306988,
 0.018647771328687668,
 0.010360945016145706,
 -0.046598006039857864,
 -0.05573476478457451,
 0.01843833737075329,
 0.0027071069926023483,
 0.03294723108410835,
 0.02504689060151577,
 -0.03142755106091499,
 -0.022691987454891205,
 0.03483140096068382,
 -0.05341877415776253,
 0.04222894087433815,
 -0.01728733628988266,
 -0.04929398000240326,
 0.00046024261973798275,
 -0.044817082583904266,
 0.0034655649214982986,
 -0.03304927796125412,
 -0.0016231774352490902,
 -0.04087826982140541,
 0.01253503654152155,
 -0.030864598229527473,
 -0.013328468427062035,
 0.012476014904677868,
 -0.037187058478593826,
 0.006219789385795593,
 -0.013966446742415428,
 0.01565675437450409,
 0.0035794111900031567,
 -0.005585063714534044,
 0.006055895704776049,
 0.00048314392915926874,
 -0.010381164960563183,
 -0.018931696191430092,
 0.012918422929942608,
 0.014919551089406013,
 -0.015488061122596264,

In [68]:
# Create a list of lists with shape (512, 128)


# Define the directory where you want to save the text files
output_directory = "ageWeights"

# Create the output directory if it doesn't exist
import os
os.makedirs(output_directory, exist_ok=True)

# Write each list to a separate text file


# Write each list to a separate text file
for i, sublist in enumerate(ageWeights):
    # Define the file name with leading zeros
    file_name = f"{i:03d}.txt"

    # Write each value in the sublist on a new line
    with open(os.path.join(output_directory, file_name), "w") as file:
        for value in sublist:
            file.write(f"{value}\n")

print("Files saved successfully.")


Files saved successfully.


In [32]:
count=1
layer1Bias=[]
for param in model.parameters(): 
    print(param.shape)
    if(count==2):
       layer1Bias = param.tolist()
       break
    count=count+1
print(len(layer1Bias),len(layer1Bias[0]))

torch.Size([512, 128])
torch.Size([512])


TypeError: object of type 'float' has no len()

In [72]:
output_directory=""
file_name = "ageBias.txt"

with open(os.path.join(output_directory, file_name), "w") as file:
        for value in ageBias:
            file.write(f"{value}\n")